In [ ]:
from maap.maap import MAAP
maap = MAAP(maap_host='api.maap-project.org')

This code is adapted from Amber Liang for use on the MAAP to match Protected and Non Protected areas

In [1]:
library("devtools")

Loading required package: usethis



In [4]:
#IN TERMINAL:
#mamba install -c conda-forge r-rlemon r-svd r-sparsem r-survival
install.packages('RItools')

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



In [ ]:
#For optmatch, paste into terminal:
#First time only, run this first:
#wget -O optmatch_0.10.6.tar.gz https://packagemanager.posit.co/cran/__linux__/bullseye/latest/src/contrib/optmatch_0.10.6.tar.gz?r_version=4.3&arch=x86_64

#R CMD INSTALL optmatch_0.10.6.tar.gz


In [5]:
#FIRST ACTIVATE gedi ENVIRONMENT

#Use mamba install -c conda-forge r-____ to install packages

#!/usr/bin/env Rscript

# This global processing script is derived from the global processing notebook 
#the input can be the iso3 code (3-character) for one or multiple countries 

options(warn=-1)
options(dplyr.summarise.inform = FALSE)

packages <- c("sp","rgdal","sf","rgeos","dplyr","plyr","ggplot2","raster","mapview","stringr",
              "maptools","gridExtra","lattice","MASS","foreach","doParallel","RItools","optmatch",
              "rlang","tidyr","magrittr","viridis","ggmap","Hmisc","hrbrthemes","spatialEco","bit64","randomForest", "modelr")
#as.factor(paste0('r-', packages," "))

#mamba install -c conda-forge r-sp r-rgdal r-sf r-rgeos r-dplyr r-plyr r-ggplot2 r-raster r-mapview r-stringr r-maptools r-gridExtra r-lattice r-MASS r-foreach r-doParallel r-rlang r-tidyr r-magrittr r-viridis r-ggmap r-Hmisc r-hrbrthemes r-spatialEco r-bit64 r-randomForest r-modelr


package.check <- lapply(packages, FUN = function(x) {
    suppressPackageStartupMessages(library(x, character.only = TRUE))
})


In [6]:
args = commandArgs(trailingOnly=TRUE)
if (length(args)==0) {
  stop("At least one argument must be supplied (input file).n", call.=FALSE)
} else if (length(args)>=1) {
  
  iso3 <- args[1]  #country to process
  gediwk <- args[2]   #the # of weeks GEDI data to use
  mproc <- as.integer(args[3])#the number of cores to use for macthing 
}

In [7]:
cat("Step 0: Loading global variables for", iso3,"with wk", gediwk, "data \n")

Step 0: Loading global variables for /projects/.local/share/jupyter/runtime/kernel-ea5a9be3-0922-4096-869f-eef1d7c6abe0.json with wk NA data 


In [8]:
f.path <- "https://maap-ops-workspace.s3.amazonaws.com/shared/abarenblitt/GEDI_PA/Matching_Layers/AGO"
matching_tifs <- c("wwf_biomes","wwf_ecoreg","lc2000","d2roads", "dcities","dem","pop_cnt_2000","pop_den_2000","slope", "tt2cities_2000", "wc_prec_1990-1999",
                   "wc_tmax_1990-1999","wc_tavg_1990-1999","wc_tmin_1990-1999" )

In [9]:
list.files(path="f.path", pattern=NULL, all.files=FALSE, 
    full.names=FALSE)

character(0)

In [10]:
ecoreg_key <- read.csv(paste(f.path,"wwf_ecoregions_key.csv",sep=""))

ERROR: Error in file(file, "rt"): cannot open the connection to 'https://maap-ops-workspace.s3.amazonaws.com/shared/abarenblitt/GEDI_PA/Matching_Layers/AGOwwf_ecoregions_key.csv'


In [18]:
ecoreg_key <- read.csv(paste(f.path,"wwf_ecoregions_key.csv",sep=""))
allPAs <- readRDS(paste(f.path,"WDPA_shapefiles/WDPA_polygons/",iso3,"_PA_poly.rds",sep=""))
MCD12Q1 <- raster(paste(f.path,"GEDI_ANCI_PFT_r1000m_EASE2.0_UMD_v1_projection_defined_6933.tif",sep=""))
projection(MCD12Q1) <- sp::CRS(paste("+init=epsg:",6933,sep=""))
world_region <- raster(paste(f.path,"GEDI_ANCI_CONTINENT_r1000m_EASE2.0_UMD_v1_revised_projection_defined_6933.tif",sep=""))
projection(world_region) <- sp::CRS(paste("+init=epsg:",6933,sep=""))
adm <- readOGR(paste(f.path,"WDPA_countries/shp/",iso3,".shp",sep=""),verbose=F)
adm_prj <- spTransform(adm, "+init=epsg:6933") 
load("/shared-buckets/abarenblitt/GEDI_PA/Matching_Layers/AGO/rf_noclimate.RData")
source("/shared-buckets/abarenblitt/GEDI_PA/Matching_Layers/AGO/matching_func.R")

ERROR: Error in file(file, "rt"): cannot open the connection
